In [1]:
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv('../../data/train.csv')
# Drop column with mostly empty values.
train = train.drop(['VAR_0205', 'VAR_0206', 'VAR_0207', 'VAR_0208', 'VAR_0209', 'VAR_0213', 'VAR_0840'], 1)
# Drop date columns for now...
train = train.drop(['VAR_0073', 'VAR_0075', 'VAR_0156', 'VAR_0157', 'VAR_0158', \
                    'VAR_0159', 'VAR_0166', 'VAR_0167', 'VAR_0168', 'VAR_0169', \
                    'VAR_0176', 'VAR_0177', 'VAR_0178', 'VAR_0179', 'VAR_0204', 'VAR_0217'], 1)
# Drop ID and target column for now...
target = train['target']
train = train.drop(['ID', 'target'], 1)
# feature = [x for x in train.columns if x not in ['ID', 'target']]

/Library/Python/2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (8,9,10,11,12,43,157,196,214,225,228,229,231,235,238) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
num_feat = train.dtypes[train.dtypes == 'float64'].index.values.tolist()
cat_feat = train.dtypes[train.dtypes == 'object'].index.values.tolist()
int_feat = train.dtypes[train.dtypes == 'int64'].index.values.tolist()
print len(num_feat), len(cat_feat)
print len(int_feat)

470 35
1404


In [8]:
# A function that fills in missing values with most frequent value, median, or custom value.
def feature_process(process_func='None', most_frequent_func=False, median_func=False, col_name=[]):
    print 'processing features...'
    for column in col_name:
        print column
        if median_func:
            process_func = train[column].median()
        if most_frequent_func:
            process_func = train[column].value_counts().index.values[0]
        train.loc[train[column].isnull(), column] = process_func

In [13]:
# Testing more efficient functional programming to fill in missing values.
f_mean = lambda df, column: df[column].mean()
f = lambda df, column: df[column].isnull()
print f(train, num_feat)

VAR_0006         0.693907
VAR_0007         0.513160
VAR_0013         0.693907
VAR_0014         0.494493
VAR_0015         0.476763
VAR_0016         1.447598
VAR_0017         0.496366
VAR_0018         0.000000
VAR_0019         0.000000
VAR_0020         0.000000
VAR_0021         0.000000
VAR_0022         0.000000
VAR_0023         0.000000
VAR_0024         0.000000
VAR_0025         0.000000
VAR_0026         0.000000
VAR_0027         0.000000
VAR_0028         0.000000
VAR_0029         0.000000
VAR_0030         0.000000
VAR_0031         0.000000
VAR_0032         0.000000
VAR_0033         1.103813
VAR_0034         0.453439
VAR_0035         0.378288
VAR_0036         0.340389
VAR_0037         0.297262
VAR_0038         0.000000
VAR_0039         0.000000
VAR_0040         0.000000
                ...      
VAR_0502         0.001129
VAR_0503         0.129018
VAR_0504         0.229764
VAR_0505         0.203765
VAR_0506         0.168308
VAR_0507         0.001538
VAR_0508         0.004566
VAR_0509    

In [20]:
# Testing efficiency

import pandas as pd
import numpy as np

np.random.seed([3,1415])
df = pd.DataFrame(np.random.choice((2, np.nan), (5, 5)))
print df

     0    1    2    3    4
0  2.0  NaN  2.0  NaN  2.0
1  2.0  NaN  2.0  NaN  NaN
2  NaN  NaN  2.0  NaN  NaN
3  2.0  NaN  2.0  2.0  2.0
4  2.0  2.0  2.0  2.0  2.0


In [25]:
def pir1(df, f):
    print df, '\n'
    dfi = df.stack(dropna=False).index.to_series().unstack() # Gets the index(i, j) of each value
    print '\n', df.combine_first(dfi.applymap(lambda x: f(*x)))
    return df.combine_first(dfi.applymap(lambda x: f(*x)))

def pir2(df, f):
    dfc = df.copy()
    for i in dfc.index:
        for j in dfc.columns:
            dfv = df.get_value(i, j)
            dfc.at[i, j] = dfv if pd.notnull(dfv) else f(i, j)
    return dfc


f = lambda i, j: i ** 2 - np.sqrt(abs(j))
f_mean = lambda df, column: df[column].mean()

def using_meshgrid(df, f):
    I, J = np.meshgrid(df.index, df.columns, sparse=True, indexing='ij')
    print pd.DataFrame(f_mean(df, df.columns), index=df.index, columns=df.columns)
    
    return df.combine_first(pd.DataFrame(f(I, J), index=df.index, columns=df.columns))

In [26]:
%%timeit
using_meshgrid(df, f)

ValueError: Shape of passed values is (1, 5), indices imply (5, 5)